## Point cloud query by time - brute force

In [1]:
import numpy as np
import pandas as pd
from laspy.file import File

line='115601'
src_dir = '../data/laz'

!mkdir -p tmp
!grep '\<.*pc.*'$line'.*\>' ../data/metadata/d15-bitstreams.json | tr -d ',' | xargs wget -P tmp -nd --no-check-certificate -P tmp
!unzip tmp/*.zip -d tmp
!rm -rf tmp/*.zip
!mkdir -p $src_dir
!find tmp -type f -exec mv {} $src_dir \;
!rm -rf tmp
!ls $src_dir

In [2]:
def get_points_by_time_brute_force(pcloud, t, tolerance = 1/1e6):
    full_idx = np.arange(len(pcloud))
    if isinstance(t, list):
        mask = (pcloud.gps_time >= t[0]) & (pcloud.gps_time < t[1])
    else:
        diff =  np.abs(pcloud.gps_time - t)
        diff_min = np.min(diff)
        if diff_min > tolerance:
            return None
        mask = diff == diff_min
    
    idx = full_idx[mask]
    
    x = pcloud.get_x_scaled()
    y = pcloud.get_y_scaled()
    z = pcloud.get_z_scaled()
    
    result = pd.DataFrame( 
        {
            'point' : [pcloud[i] for i in idx],
            'x' : [x[i] for i in idx],
            'y' : [y[i] for i in idx],
            'z' : [z[i] for i in idx]
        }, index = idx
    )
           
    return result

### Parse a point cloud

In [3]:
pcloud = File("../data/laz/F_150326_115601.laz", mode = "r")

### Match query

Search for points that match a given timestamp

In [4]:
timestamp = 388567044563/1000000 # in seconds
%time pts = get_points_by_time_brute_force(pcloud, timestamp)
print('number of points found: {}'.format(len(pts)))
pts

CPU times: user 308 ms, sys: 9.35 ms, total: 318 ms
Wall time: 316 ms
number of points found: 2


,point,x,y,z
0,<laspy.util.Point object at 0x7fd6180cb518>,315459.722,233580.778,-61.811
13291790,<laspy.util.Point object at 0x7fd6180cb5c0>,315434.282,233547.335,10.737


### Range query

Search for points whose timestamps are within a range

In [5]:
t0 = 388567044563/1000000
t1 = t0 + .01
timestamp = [t0, t1] 
%time pts = get_points_by_time_brute_force(pcloud, timestamp)
print('number of points found: {}'.format(len(pts)))
pts

CPU times: user 373 ms, sys: 0 ns, total: 373 ms
Wall time: 372 ms
number of points found: 2842


,point,x,y,z
0,<laspy.util.Point object at 0x7fd64cf5bf98>,315459.722,233580.778,-61.811
2256598,<laspy.util.Point object at 0x7fd64cf5bf60>,315231.562,233283.255,13.691
2256599,<laspy.util.Point object at 0x7fd64cf5bfd0>,315231.652,233283.372,13.670
2256600,<laspy.util.Point object at 0x7fd64cf6f048>,315231.755,233283.506,13.676
2256601,<laspy.util.Point object at 0x7fd64cf6f080>,315231.930,233283.734,13.931
...,...,...,...,...
13426600,<laspy.util.Point object at 0x7fd64ce045f8>,315437.962,233552.053,31.996
13426601,<laspy.util.Point object at 0x7fd64ce04630>,315438.341,233552.550,31.300
13426602,<laspy.util.Point object at 0x7fd64ce04668>,315438.317,233552.519,31.621
13426603,<laspy.util.Point object at 0x7fd64ce046a0>,315438.418,233552.649,31.655


### References 

* [Query types](https://docs.couchbase.com/server/current/fts/fts-query-types.html)